Oskar Lundqvist; osauli-0@student.ltu.se

Filip Renberg; filren-0@student.ltu.se

We are gonna do a simple lab where we train a model using pytorch

In [ ]:
import torch
import torch.nn as nn
import numpy as np

We create our multi-layer neural network in pytorch

In [ ]:
from collections import OrderedDict

model = nn.Sequential(OrderedDict([
    ('dense1', nn.Linear(764, 100)),
    ('act1', nn.ReLU()),
    ('dense2', nn.Linear(100, 50)),
    ('act2', nn.ReLU()),
    ('output', nn.Linear(50, 10)),
    ('outact', nn.Sigmoid()),
]))

We are gonna use the mnist dataset of digits imported from keras like we did in lab 1 and lab 4.

In [7]:
import keras
import tensorflow
from keras.datasets import mnist


(train_data, train_labels), (test_data, test_labels) = mnist.load_data()

ModuleNotFoundError: No module named 'tensorflow.python'